In [ ]:
import csv
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler
from transformers import AdamW
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import random

In [ ]:
# Define the path to the input CSV file
input_file = '/content/drive/MyDrive/ir-project/input.csv'

# Define the path to the output CSV file
output_file = '/content/drive/MyDrive/ir-project/output.csv'

# Create a list to store the scraped data
scraped_data = []

# Read the input CSV file
with open(input_file, 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row
    for row in reader:
        link = row[0]
        name = row[1]

        try:
          # Make a request to the Wikipedia page
          response = requests.get(link)
          soup = BeautifulSoup(response.text, 'html.parser')

          # Extract the title and text from the page
          title = soup.find('h1', {'id': 'firstHeading'}).text
          content_div = soup.find('div', {'id': 'mw-content-text'})
          paragraphs = content_div.find_all('p')
          text = '\n'.join([p.text for p in paragraphs])

          # Append the scraped data to the list
          scraped_data.append({'Title': name, 'Text': text})

        except requests.exceptions.RequestException as e:
          print(f"Error: {link}")
          print(e)
          continue

# Write the scraped data to the output CSV file
with open(output_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Text'])
    writer.writeheader()
    writer.writerows(scraped_data)



Error: Hormone - Wikipedia
Invalid URL 'Hormone - Wikipedia': No scheme supplied. Perhaps you meant http://Hormone - Wikipedia?


In [ ]:
file1 = open("/content/drive/MyDrive/ir-project/output.csv", 'r')
reader = csv.reader(file1)


In [ ]:
extracted_data = []

for row in reader:

  title = row[0]
  text = row[1]

  data = {
      "title": title,
      "text": text,
  }

  extracted_data.append(data)

print(len(extracted_data))

if extracted_data: print("ok")
train_data = extracted_data

90
ok


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

input_ids = []
attention_masks = []
labels = []
processed_train_data = []

for article in train_data:
    title = article['title']
    text = article['text']


    # Tokenize the text and add special tokens
    encoded_text = tokenizer.encode_plus(
        title,
        text,
        add_special_tokens=True,
        max_length=512,  # Adjust the max length as needed
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'  # Return PyTorch tensors
    )

    # Extract the input IDs and attention masks
    input_ids.append(encoded_text['input_ids'])
    attention_masks.append(encoded_text['attention_mask'])


    processed_train_data.append({
      "title": title,
      "text": text,
      "input_ids": input_ids,
      'attention_masks': attention_masks,

    })



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
# stop words for filtering the category lables
stopwords =[
    'wikipedia',
    'a',
    'about',
    'above',
    'across',
    'after',
    'afterwards',
    'again',
    'against',
    'ain',
    'all',
    'almost',
    'alone',
    'along',
    'already',
    'also',
    'although',
    'always',
    'am',
    'among',
    'amongst',
    'amoungst',
    'amount',
    'an',
    'and',
    'another',
    'any',
    'anyhow',
    'anyone',
    'anything',
    'anyway',
    'anywhere',
    'are',
    'aren',
    'around',
    'as',
    'at',
    'back',
    'be',
    'became',
    'because',
    'become',
    'becomes',
    'becoming',
    'been',
    'before',
    'beforehand',
    'behind',
    'being',
    'below',
    'beside',
    'besides',
    'between',
    'beyond',
    'bill',
    'both',
    'bottom',
    'but',
    'by',
    'call',
    'can',
    'cannot',
    'cant',
    'co',
    'con',
    'could',
    'couldn',
    'couldnt',
    'cry',
    'd',
    'de',
    'describe',
    'detail',
    'did',
    'didn',
    'do',
    'does',
    'doesn',
    'doing',
    'don',
    'done',
    'down',
    'due',
    'during',
    'each',
    'eg',
    'eight',
    'either',
    'eleven',
    'else',
    'elsewhere',
    'empty',
    'enough',
    'etc',
    'even',
    'ever',
    'every',
    'everyone',
    'everything',
    'everywhere',
    'except',
    'few',
    'fifteen',
    'fify',
    'fill',
    'find',
    'fire',
    'first',
    'five',
    'for',
    'former',
    'formerly',
    'forty',
    'found',
    'four',
    'from',
    'front',
    'full',
    'further',
    'get',
    'give',
    'go',
    'had',
    'hadn',
    'has',
    'hasn',
    'hasnt',
    'have',
    'haven',
    'having',
    'he',
    'hence',
    'her',
    'here',
    'hereafter',
    'hereby',
    'herein',
    'hereupon',
    'hers',
    'herself',
    'him',
    'himself',
    'his',
    'how',
    'however',
    'hundred',
    'i',
    'ie',
    'if',
    'in',
    'inc',
    'indeed',
    'interest',
    'into',
    'is',
    'isn',
    'it',
    'its',
    'itself',
    'just',
    'keep',
    'last',
    'latter',
    'latterly',
    'least',
    'less',
    'll',
    'ltd',
    'm',
    'ma',
    'made',
    'many',
    'may',
    'me',
    'meanwhile',
    'might',
    'mightn',
    'mill',
    'mine',
    'more',
    'moreover',
    'most',
    'mostly',
    'move',
    'much',
    'must',
    'mustn',
    'my',
    'myself',
    'name',
    'namely',
    'needn',
    'neither',
    'never',
    'nevertheless',
    'next',
    'nine',
    'no',
    'nobody',
    'none',
    'noone',
    'nor',
    'not',
    'nothing',
    'now',
    'nowhere',
    'o',
    'of',
    'off',
    'often',
    'on',
    'once',
    'one',
    'only',
    'onto',
    'or',
    'other',
    'others',
    'otherwise',
    'our',
    'ours',
    'ourselves',
    'out',
    'over',
    'own',
    'part',
    'per',
    'perhaps',
    'please',
    'put',
    'rather',
    're',
    's',
    'same',
    'see',
    'seem',
    'seemed',
    'seeming',
    'seems',
    'serious',
    'several',
    'shan',
    'she',
    'should',
    'shouldn',
    'show',
    'side',
    'since',
    'sincere',
    'six',
    'sixty',
    'so',
    'some',
    'somehow',
    'someone',
    'something',
    'sometime',
    'sometimes',
    'somewhere',
    'still',
    'such',
    'system',
    't',
    'take',
    'ten',
    'than',
    'that',
    'the',
    'their',
    'theirs',
    'them',
    'themselves',
    'then',
    'thence',
    'there',
    'thereafter',
    'thereby',
    'therefore',
    'therein',
    'thereupon',
    'these',
    'they',
    'thick',
    'thin',
    'third',
    'this',
    'those',
    'though',
    'three',
    'through',
    'throughout',
    'thru',
    'thus',
    'to',
    'together',
    'too',
    'top',
    'toward',
    'towards',
    'twelve',
    'twenty',
    'two',
    'un',
    'under',
    'until',
    'up',
    'upon',
    'us',
    've',
    'very',
    'via',
    'was',
    'wasn',
    'we',
    'well',
    'were',
    'weren',
    'what',
    'whatever',
    'when',
    'whence',
    'whenever',
    'where',
    'whereafter',
    'whereas',
    'whereby',
    'wherein',
    'whereupon',
    'wherever',
    'whether',
    'which',
    'while',
    'whither',
    'who',
    'whoever',
    'whole',
    'whom',
    'whose',
    'why',
    'will',
    'with',
    'within',
    'without',
    'won',
    'would',
    'wouldn',
    'y',
    'yet',
    'you',
    'your',
    'yours',
    'yourself',
    'yourselves'
]

In [ ]:


# Extract the text from the documents
text_data = [document['text'] for document in train_data]

# Create TF-IDF vectors from the text data
vectorizer = TfidfVectorizer(stop_words=stopwords)
tfidf_matrix = vectorizer.fit_transform(text_data)

# Perform K-means clustering
num_clusters = random.randint(10, 20)  # Random number of desired categories
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Assign cluster labels to the documents
cluster_labels = kmeans.labels_

# Create a dictionary to store documents for each category
categories = {i: [] for i in range(num_clusters)}

# Assign each document to its corresponding category
for i, document in enumerate(train_data):
    category = cluster_labels[i]
    categories[category].append(document)

# Keyword Extraction and Tag Generation
tags = {}

for category, documents in categories.items():
    all_text = [document['text'] for document in documents]
    combined_text = ' '.join(all_text)

    # Create TF-IDF vectors for the category
    category_tfidf = vectorizer.transform([combined_text])

    # Extract top keywords based on TF-IDF scores
    top_keywords = [keyword for keyword, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x: category_tfidf[0, x[1]], reverse=True)]

    # Randomly select 5 to 8 tags
    num_tags = random.randint(5, 8)
    selected_tags = random.sample(top_keywords, min(num_tags, len(top_keywords)))

    # Store the selected tags for the category
    tags[category] = selected_tags

# Print the tags for each category
for category, keywords in tags.items():
    print(f"Category {category}:")
    print("Tags:", keywords)
    print()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Category 0:
Tags: ['complaints', '1762', '26', 'underfunding', 'cheaper', 'lil']

Category 1:
Tags: ['unbeknownst', 'antonio', 'seaboard', 'printable', 'sesame', 'kilobyte']

Category 2:
Tags: ['plasma', 'designing', 'encodings', 'charge', 'georgian', 'seventeen', 'phillip']

Category 3:
Tags: ['forsberg', 'realism', 'ornamentation', 'sippar', 'bracher', 'flash', 'widening', 'enharmonic']

Category 4:
Tags: ['organic', 'spring', 'adding', 'rulers', 'civilisation']

Category 5:
Tags: ['monasteries', 'masses', 'processions', 'formative', 'intolerable', 'pharmaceutical']

Category 6:
Tags: ['lipids', 'anomalies', 'grammar', '359', 'voor', 'utter', 'eastward']

Category 7:
Tags: ['hypertensive', 'cutters', 'circadian', 'virtualbox', 'heilman', 'brygians', 'countryside']

Category 8:
Tags: ['knew', 'duardos', 'halves', 'warnings', 'message', 'dairyban', 'sasanians']

Category 9:
Tags: ['huế', 'broszat', 'airliner', 'thorium', 'aurangabad', 'weakness']

Category 10:
Tags: ['457', 'villages',

In [ ]:

# User Query
user_query = input("Enter your query: ")

# Tokenize the user query
user_query_tfidf = vectorizer.transform([user_query])

# Compute cosine similarity between the user query and each category
category_similarities = {}
for category, documents in categories.items():
    category_text = ' '.join([document['text'] for document in documents])
    category_tfidf = vectorizer.transform([category_text])
    similarity = cosine_similarity(user_query_tfidf, category_tfidf)[0][0]
    category_similarities[category] = similarity

# Find the most similar category
most_similar_category = max(category_similarities, key=category_similarities.get)
print("Most Similar Category:", most_similar_category)
print("Similarity:", category_similarities[most_similar_category])
print("------------")
print()

# Sort the documents in the most similar category by their similarity to the query
most_similar_documents = categories[most_similar_category]
most_similar_documents.sort(key=lambda document: cosine_similarity(user_query_tfidf, vectorizer.transform([document['text']]))[0][0], reverse=True)

# Print the documents and their similarity ratings
for document in most_similar_documents:
    similarity = cosine_similarity(user_query_tfidf, vectorizer.transform([document['text']]))[0][0]
    print("Document:")
    print("Title:", document['title'])
    # print("Text:", document['text'])
    print("Similarity:", similarity)
    print()



Enter your query: 16 bit
Most Similar Category: 2
Similarity: 0.554228941539971
------------

Document:
Title: 16-bit computing - Wikipedia
Similarity: 0.8476506779319539

Document:
Title: 16-bit computing - Wikipedia
Similarity: 0.8476506779319539

Document:
Title: x86 - Wikipedia
Similarity: 0.40596398819280516

Document:
Title: Random-access memory - Wikipedia
Similarity: 0.10803248041319585

